In [ ]:
!pip install -r requirements.txt

In [3]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import os
from PIL import Image
import google.generativeai as genai
from pdf2image import convert_from_bytes
import fitz  # PyMuPDF
from PIL import Image,ImageEnhance
from IPython.display import Image as new_Image
import shutil
import joblib

**Enter your Gemini API key and path of Invoice(PDF)**

In [ ]:

API_key = input('Enter Gemini API Key : ')
uploaded_file = input('Path of Invoice(PDF) : ')


In [22]:
genai.configure(api_key=API_key)
model=genai.GenerativeModel('gemini-pro-vision')

In [23]:
def get_gemini_response(input_prompt, image_data_list, user_input):
    # Combine the input, image data, and user input into a single list
    inputs = [input_prompt]
    for image in image_data_list:
        inputs.append(image[0])
    inputs.append(user_input)
    # Call the model with the combined inputs
    response = model.generate_content(inputs)

    return response.text

def pdf_to_images(pdf_bytes, resolution=1200):
    pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")

    images = []  # List to store images for each page

    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)

        # Customize Pixmap creation for improved quality
        matrix = fitz.Matrix(resolution/300, resolution/300)
        pixmap = page.get_pixmap(matrix=matrix, colorspace="rgb")

        # Create a white canvas
        white_background = Image.new("RGB", (pixmap.width, pixmap.height), (255, 255, 255))

        # Create PIL image from the Pixmap and overlay it on the white canvas
        pil_image = Image.frombytes("RGB", (pixmap.width, pixmap.height), pixmap.samples)
        white_background.paste(pil_image, (0, 0))

        # Apply additional image enhancement if needed
        enhanced_image = ImageEnhance.Contrast(white_background).enhance(1.5)  # Example contrast enhancement

        images.append(enhanced_image)

    return images

def input_image_details(folder_path):


    folder_path = os.path.join(os.getcwd(), folder_path)

    image_parts_list=[]

    for filename in os.listdir(folder_path):
        # Check if the file exists
        image_path = os.path.join(folder_path, filename)
        if os.path.exists(image_path):
            # Read the image into bytes
            with open(image_path, "rb") as image_file:
                bytes_data = image_file.read()

            # Create a dictionary with the standardized structure
            image_parts = [
                {
                    "mime_type": "image/png",  # Assuming PNG format for simplicity
                    "data": bytes_data
                }
            ]

            image_parts_list.append(image_parts)
        else:
            raise FileNotFoundError("Image file not found.")

    return image_parts_list


def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return text

In [24]:
ocr_info = 'No OCR info is provided for this file'

if uploaded_file is not None:

    file_extension = uploaded_file.split('.')[-1].lower()

    if file_extension == 'pdf':

        with open(uploaded_file, 'rb') as file:
            pdf_bytes = file.read()

        # Convert PDF to image using PyMuPDF
        pdf_images = pdf_to_images(pdf_bytes)

        ocr_info = get_pdf_text([uploaded_file])

        # Remove existing "uploaded_images" folder if it exists
        folder_name = "uploaded_images"
        if os.path.exists(folder_name):
            shutil.rmtree(folder_name)

        # Create a new folder to save the images
        os.makedirs(folder_name)

        # Save each image in the folder with a numbered filename
        for i, pdf_image in enumerate(pdf_images):
            image_filename = f"uploaded_image_{i+1}.jpg"
            image_path = os.path.join(folder_name, image_filename)
            pdf_image.save(image_path)
    else:
        print('not a pdf ..... pls upload a pdf')


In [ ]:
file_path = r'/content/input_prompt.txt'

with open(file_path, 'r') as file:
    input_prompt = file.read()

input_prompt = input_prompt.replace('{ocr_info}', ocr_info)

with open('/content/input_prompt2.txt', 'w') as file:
    file.write(input_prompt)

while True:
    print('You : ',end='')
    user_input=input()
    print()
    folder_path = '/content/uploaded_images'  # Replace with the actual folder path
    image_data_list = input_image_details(folder_path)

    response = get_gemini_response(input_prompt, image_data_list, user_input)

    chat_history = f'User : {user_input} \nYou : {response} \n'
    file_path = r'/content/input_prompt2.txt'

    with open(file_path, 'a') as file:

        file.write(chat_history)

    input_prompt = input_prompt + chat_history

    print("Gemini : ",end='')
    print(response)
    print()